# Exemplos Aplicados

Nesta seção final, abordaremos alguns exemplos aplicados com o NumPy. Abordaremos alguns exemplos "do zero", além de usar a biblioteca scikit-learn. Esperamos que isso convença você da ideia de que usar matrizes e matemática vetorizada pode ser útil e prático.

## Regressão Linear com Hill Climbing

Embora este não seja o algoritmo mais eficiente disponível para regressão linear, é um exercício interessante que ensina conceitos aplicáveis ​​a outros problemas. Mais especificamente, vamos aprender a escalar colinas. A ideia por trás da escalar colinas é fazer ajustes aleatórios em uma solução e, se esses ajustes resultarem em uma melhoria, nós os mantemos. Fazemos isso milhares ou milhões de vezes até que a solução não melhore mais.

Uma regressão linear simples resolve os coeficientes $ \beta_0 $ e $ \beta_1 $, dados os dados de entrada $ x $ e os dados de saída $ y $. Tentamos avaliar uma relação linear entre $ x $ e $ y $ e ajustar os dados de acordo.

$$
\Large y = \Large \beta_0 + \Large \beta_1 x
$$

Vamos importar essas duas colunas de dados $ x $ e $ y $ do GitHub e salvá-las em dois arrays NumPy.

In [ ]:
import numpy as np
import pandas as pd

data = pd.read_csv("https://bit.ly/2KF29Bd").values
x = data[:,0]
y = data[:,-1]

In [ ]:
x

In [ ]:
y

Em seguida, declararemos os coeficientes beta `b` como um array NumPy que inicializará $ \beta_0 $ e $ \beta_1 $ como $ 0,0 $. Também executaremos nossa escalada de colina por 150.000 iterações (`épocas`) e iniciaremos nossa perda com um número MUITO alto. Isso fará mais sentido em breve.

In [ ]:
# Construindo o modelo
b = np.array([0.0, 0.0]) 

epocas = 150000  # Número de iterações a serem realizadas
n = float(data.shape[0])  # Número de pontos
melhor_perda = 10000000000000.0  # Inicializar com um valor bem grande

Vamos definir como objetivo uma função de perda, mais especificamente a soma dos quadrados. Para uma determinada reta com $ \beta_0 $ e $ \beta_1 $, calculamos as diferenças entre a reta e os pontos de dados fornecidos e, em seguida, elevamos ao quadrado e somamos essas diferenças. Essa soma dos quadrados é o que queremos minimizar. Para cada ajuste aleatório em $ \beta_0 $ e $ \beta_1 $, calcularemos a soma dos quadrados e verificaremos se ela foi reduzida. Caso contrário, desfazemos esses ajustes aleatórios.

Mas como ajustamos aleatoriamente $ \beta_0 $ e $ \beta_1 $? Podemos usar valores aleatórios da distribuição normal, com média 0 e desvio padrão 1. Isso criará um volume de ajustes, em sua maioria pequenos, próximos de 0, mas ocasionalmente fazemos ajustes maiores nas extremidades, nas direções positiva e negativa.

In [ ]:
for i in range(epocas):

    # Ajustar aleatoriamente "m" e "b"
    ajuste_aleatorio = np.random.normal(loc=0, scale=1, size=2)
    b += ajuste_aleatorio
    
    # Calcular a perda (loss), que é o total do erro quadrático somado
    nova_perda = ((y - (b[0] + b[1] * x)) ** 2).sum()
    
    # Se a perda melhorou, manter os novos valores. Caso contrário, reverter.
    if nova_perda < melhor_perda:
        print(f"y = {b[0]} + {b[1]}x")
        melhor_perda = nova_perda
    else:
        b -= ajuste_aleatorio
        
print(f"y = {b[0]} + {b[1]}x")

Para ver rapidamente a qualidade do ajuste, vamos usar o matplotlib para ver o quão bem a linha se ajusta aos pontos de dados.

In [ ]:
# plot 
import matplotlib.pyplot as plt
import numpy as np 

# show in chart
plt.plot(x, y, 'o') # scatterplot
plt.plot(x, b[1]*x + b[0]) # line
plt.show()

## Regressão Linear com scikit-learn

Embora a escalada possa ser usada para resolver uma variedade de problemas, a regressão linear possui alguns atalhos usando métodos de álgebra linear que abordamos [em outro curso](https://learning.anaconda.cloud/linear-algebra). Mas, por enquanto, podemos usar o scikit-learn e aproveitar matrizes NumPy para passar os dados.

Abaixo, lemos o mesmo CSV do exemplo anterior e extraímos as colunas `X` e `y`. Em seguida, ajustamos uma `LinearRegression` e obtemos os coeficientes de `coef_` e `intercept_`. No entanto, eles retornarão matrizes multidimensionais, então as achatamos.

In [ ]:
from sklearn.linear_model import LinearRegression
import pandas as pd 
import matplotlib.pyplot as plt

df = pd.read_csv("https://bit.ly/2KF29Bd", delimiter=",")

# Extrair as variáveis de entrada (todas as linhas, todas as colunas exceto a última)
x = df.values[:, :-1]

# Extrair a variável de saída (todas as linhas, última coluna)
y = df.values[:, -1]

# Ajustar uma linha aos pontos
ajuste = LinearRegression().fit(x, y)

# m = 1.7867224, b = -16.51923513
b1 = ajuste.coef_.flatten()
b0 = ajuste.intercept_.flatten()
print(f"b1 = {b1}")
print(f"b0 = {b0}")

# Mostrar no gráfico
plt.plot(x, y, 'o')  # gráfico de dispersão
plt.plot(x, b1*x+b0)  # linha de regressão
plt.show()

## Simulação de fila de clientes

Neste exemplo, simularemos clientes entrando em um banco, supermercado, etc., onde serão atendidos individualmente. Para simplificar, teremos apenas um atendente para atender a cada cliente, embora você possa adaptar isso para atender a vários atendentes. O objetivo desta simulação é verificar se uma fila se formará e ficará fora de controle, e teoricamente poderíamos usar isso para prever o tempo médio de espera dos clientes.

Lembre-se de que estudamos diversas distribuições de probabilidade. A distribuição normal pode fazer sentido para o tempo de atendimento ao cliente, assumindo que o tempo de atendimento segue uma distribuição normal. Mas e quanto ao fluxo de clientes na loja? A distribuição exponencial modelará o tempo decorrido entre a entrada de cada cliente.

Vamos construir a simulação abaixo usando a distribuição normal e a distribuição exponencial. Os clientes levarão em média 3 minutos para serem atendidos, com um desvio padrão de 0,5 minuto. Modelaremos 20 clientes, em média, chegando a cada hora, mas para sermos consistentes em minutos, isso equivale a US$ 1/3 de um cliente a cada minuto. Executaremos a simulação para os primeiros 100 clientes.

Observe que isso é um pouco complexo, mas a ideia é usar essas duas distribuições para criar uma simulação "realista". Execute a simulação e observe o resultado antes de mergulhar no código em si.

In [ ]:
import numpy as np
from numpy.random import normal, exponential

np.random.seed(0)  # Usar semente aleatória para executar simulações "aleatórias" idênticas

tempo_medio_caixa = 3  # minutos
desvio_padrao_caixa = 0.5  # minutos
taxa_media_chegada = 20 / 60  # clientes por minuto
qtd_clientes = 100

# Tempos entre chegadas dos clientes (tempo entre cada cliente em relação ao anterior)
tempos_entre_clientes = exponential(scale=1/taxa_media_chegada, size=qtd_clientes+2)  # precisa adicionar 2 para evitar erro de índice

# Horário de chegada de cada cliente (minutos desde o início da simulação)
horarios_chegada_clientes = np.cumsum(tempos_entre_clientes)

# Tempos de atendimento dos clientes
tempos_atendimento_clientes = normal(loc=tempo_medio_caixa, scale=desvio_padrao_caixa, size=qtd_clientes+2)  # adicionar 2 para evitar erro de índice

# Começar o tempo no 0, mas pular para a chegada do primeiro cliente
# e rastrear se há cliente sendo atendido e quais estão esperando
tempo_atual = horarios_chegada_clientes[0]
clientes_esperando = []

cliente_chegou_i = 0
cliente_sendo_processado_i = 0
inicio_atendimento_cliente = horarios_chegada_clientes[0]

# Processar clientes até que todos tenham chegado
while cliente_chegou_i < qtd_clientes:

    # Horário de chegada do cliente sendo atendido
    chegada_cliente_processando = horarios_chegada_clientes[cliente_sendo_processado_i]

    # Horário programado de término do atendimento do cliente
    fim_cliente_processando = inicio_atendimento_cliente + \
                              tempos_atendimento_clientes[cliente_sendo_processado_i]

    # Próxima chegada de cliente
    def proxima_chegada_cliente(): return horarios_chegada_clientes[cliente_chegou_i+1]

    # VERIFICAR QUAL EVENTO OCORREU COMPARANDO OS TEMPOS
    proximo_evento = None

    # Se for o primeiro cliente
    if tempo_atual == inicio_atendimento_cliente:
        print(f"{tempo_atual}: CLIENTE {cliente_chegou_i} CHEGOU, SEM FILA, ATENDIMENTO IMEDIATO")
        proximo_evento = np.min([fim_cliente_processando, proxima_chegada_cliente()])

    # Se um cliente chegar
    elif tempo_atual == proxima_chegada_cliente():
        cliente_chegou_i += 1  # Incrementa o índice de clientes que chegaram

        # Se não houver fila e o cliente que chegou for o próximo a ser atendido
        if cliente_sendo_processado_i == cliente_chegou_i:
            inicio_atendimento_cliente = tempo_atual
            fim_cliente_processando = inicio_atendimento_cliente + tempos_atendimento_clientes[cliente_sendo_processado_i]

            print(f"{tempo_atual}: CLIENTE {cliente_chegou_i} CHEGOU, SEM FILA, ATENDIMENTO IMEDIATO")
        else:
            # Senão, há fila e o cliente precisa esperar
            clientes_esperando.append(cliente_chegou_i)
            print(f"{tempo_atual}: CLIENTE {cliente_chegou_i} CHEGOU, ADICIONANDO À FILA {clientes_esperando}")

        # Próximo evento: cliente atual terminando ou próximo cliente chegando
        proximo_evento = np.min([fim_cliente_processando, proxima_chegada_cliente()])

    # Se um cliente terminar o atendimento
    elif tempo_atual == fim_cliente_processando:

        # Se a fila não estiver vazia, atender o próximo da fila
        if clientes_esperando:
            clientes_esperando.pop(0)
            print(f"{tempo_atual}: CLIENTE {cliente_sendo_processado_i} TERMINOU, CLIENTE {cliente_sendo_processado_i + 1}"
                  f" REMOVIDO DA FILA {clientes_esperando}")

            inicio_atendimento_cliente = tempo_atual

            # Próximo evento: este cliente terminando ou próximo cliente chegando
            proximo_evento = np.min([
                inicio_atendimento_cliente + tempos_atendimento_clientes[cliente_sendo_processado_i + 1],
                proxima_chegada_cliente()
            ])
        else:
            # Se a fila estiver vazia, esperar o próximo cliente
            print(f"{tempo_atual}: CLIENTE {cliente_sendo_processado_i} TERMINOU, ESPERANDO O CLIENTE {cliente_sendo_processado_i + 1}")
            proximo_evento = proxima_chegada_cliente()

        cliente_sendo_processado_i += 1  # Próximo cliente a ser processado

    # Avançar para o próximo evento
    tempo_atual = proximo_evento

Ao executar a simulação acima, você verá que uma fila se forma irremediavelmente após um tempo suficiente. Isso deve indicar que outro caixa pode ser necessário para atender os clientes! Você também pode experimentar tempos de atendimento mais curtos ou intervalos maiores entre os clientes, e descobrirá que há um equilíbrio ideal em algum momento em que o atendimento acompanha a fila.

## Rede Neural com scikit-learn

Vamos dar uma olhada em um último exemplo de uso do NumPy com o scikit-learn. Vamos primeiro importar algumas informações.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix

Vamos introduzir o conjunto de dados MNIST. Observe que cada linha é uma imagem de um dígito escrito à mão, embora isso ainda não seja totalmente óbvio. Mas vemos que cada coluna contém um valor de pixel para cada imagem/linha. A única exceção é a última coluna, que é o rótulo do dígito que a imagem representa, entre 0 e 9.

In [ ]:
df = pd.read_csv('https://bit.ly/3ilJc2C', compression='zip', delimiter=",")
df

Esses valores de pixel representam o quão escuro ele está, em uma escala entre 0 e 255. Vamos dividir por 255 para que fique entre 0 e 1. Vamos também pegar as colunas de entrada de pixels como `x` e a coluna de saída dos rótulos `y`.

In [ ]:
x = df.values[:, :-1] / 255.0
y = df.values[:, -1]

Em aprendizado de máquina, é uma boa prática reservar parte dos dados como conjunto de dados de teste (por exemplo, 1/3 dos dados) e usar os dados restantes como conjunto de dados de treinamento (por exemplo, 2/3). Dessa forma, podemos usar os dados de teste posteriormente para avaliar o desempenho do modelo em dados que ele ainda não viu. O método `train_test_split()` fará isso por nós e retornará quatro matrizes NumPy que servirão como conjuntos de dados de treinamento/teste.

In [ ]:
# Separar dados de treino e teste
# cada classe é proporcionalmente representada em ambos os conjuntos
X_treino, X_teste, Y_treino, Y_teste = train_test_split(x, y,
    test_size=1/3, random_state=10)

Por fim, criaremos uma rede neural de classificação `MLPClassifier` a partir do scikit-learn. Passaremos os dados `X_train` e `Y_train` para o modelo, usaremos 100 nós ocultos em uma única camada oculta e usaremos uma função de ativação logística para a camada oculta. Se você não está familiarizado com aprendizado de máquina ou redes neurais, [há uma aula no Anaconda que ensina esse tópico](https://learning.anaconda.cloud/getting-started-with-ai-ml).

Vamos treinar o modelo e, em seguida, avaliar a precisão dos dados de teste. Podemos ir um passo além e criar uma matriz de confusão, que rastreia com que frequência cada dígito foi identificado corretamente e, quando não foram, quais dígitos foram classificados incorretamente. A própria matriz de confusão será retornada como um array NumPy.

In [ ]:
nn = MLPClassifier(solver='sgd',
                   hidden_layer_sizes=(100, ),
                   activation='logistic',
                   max_iter=480,
                   learning_rate_init=.1)

nn.fit(X_treino, Y_treino)

print(f"Acurácia no conjunto de teste: {nn.score(X_teste, Y_teste)}")

matriz_confusao = confusion_matrix(y_true=Y_teste, y_pred=nn.predict(X_teste))
print(matriz_confusao)


Isso deve nos dar muitos exemplos para refletir. Espero que você perceba que o NumPy é um bloco de construção para executar muitas tarefas e trabalhar com muitas bibliotecas, como o scikit-learn.

## Exercício

Complete o código abaixo para realizar uma regressão linear neste conjunto de dados no GitHub. A coluna da esquerda é a variável de entrada `x` e a coluna da direita é a variável de saída `y`.

Complete substituindo os pontos de interrogação `?` abaixo, incluindo a extração dos coeficientes.

In [ ]:
from sklearn.linear_model import LinearRegression
import pandas as pd 
import matplotlib.pyplot as plt

url = r"https://raw.githubusercontent.com/thomasnield/machine-learning-demo-data/master/regression/linear_normal.csv"
df = pd.read_csv(url, delimiter=",")

# Extrair variáveis de entrada (todas as linhas, todas as colunas exceto a última)
x = ?

# Extrair a coluna de saída (todas as linhas, última coluna)
y = ?

# Ajustar uma linha aos pontos
ajuste = LinearRegression().fit(?, ?)

b1 = ajuste.coef_.?
b0 = ajuste.intercept_.?
print(f"b1 = {b1}")
print(f"b0 = {b0}")

# Mostrar no gráfico
plt.plot(x, y, 'o')  # gráfico de dispersão
plt.plot(x, b1*x+b0)  # linha de regressão
plt.show()

### RESPOSTA A BAIXO

|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
v 

In [ ]:
from sklearn.linear_model import LinearRegression
import pandas as pd 
import matplotlib.pyplot as plt

url = r"https://raw.githubusercontent.com/thomasnield/machine-learning-demo-data/master/regression/linear_normal.csv"
df = pd.read_csv(url, delimiter=",")

# Extrair variáveis de entrada (todas as linhas, todas as colunas exceto a última)
x = df.values[:, :-1]

# Extrair variável de saída (todas as linhas, última coluna)
y = df.values[:, -1]

# Ajustar uma linha aos pontos
ajuste = LinearRegression().fit(x, y)

b1 = ajuste.coef_.flatten()
b0 = ajuste.intercept_.flatten()
print(f"b1 = {b1}")
print(f"b0 = {b0}")

# Mostrar no gráfico
plt.plot(x, y, 'o')  # Gráfico de dispersão
plt.plot(x, b1 * x + b0)  # Linha de regressão
plt.show()